## 1. Importing Libraries 💽

In [2]:
import pandas as pd
import numpy as np
import joblib

pd.set_option('display.max_columns', None)

## 2. DATA LOADING 💽

In [3]:
df = pd.read_csv('/content/RTA Dataset.csv')

In [4]:
#Display top 10 rows of dataframe df
df.head(10)

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Area_accident_occured,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,No defect,Residential areas,NaN,Tangent road with flat terrain,No junction,Asphalt roads,Dry,Daylight,Normal,Collision with roadside-parked vehicles,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,No defect,Office areas,Undivided Two way,Tangent road with flat terrain,No junction,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,No defect,Recreational areas,other,NaN,No junction,Asphalt roads,Dry,Daylight,Normal,Collision with roadside objects,2,2,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,No defect,Office areas,other,Tangent road with mild grade and flat terrain,Y Shape,Earth roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,No defect,Industrial areas,other,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Darkness - lights lit,Normal,Vehicle with vehicle collision,2,2,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
5,14:15:00,Friday,31-50,Male,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y Shape,NaN,Dry,Daylight,Normal,Vehicle with vehicle collision,1,1,U-Turn,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Overloading,Slight Injury
6,17:30:00,Wednesday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Owner,NaN,No defect,Residential areas,Undivided Two way,Tangent road with flat terrain,Crossing,NaN,Dry,Daylight,Normal,Vehicle with vehicle collision,1,1,Moving Backward,Driver or rider,Female,18-30,3,Driver,Normal,Not a Pedestrian,Other,Slight Injury
7,17:20:00,Friday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Governmental,Above 10yr,No defect,Residential areas,other,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Daylight,Normal,Vehicle with vehicle collision,2,1,U-Turn,na,na,na,na,NaN,Normal,Not a Pedestrian,No priority to vehicle,Slight Injury
8,17:20:00,Friday,18-30,Male,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,1-2yr,No defect,Industrial areas,other,Tangent road with flat terrain,Y Shape,Earth roads,Dry,Daylight,Normal,Collision with roadside-parked vehicles,2,1,Going straight,Pedestrian,Male,Under 18,3,Driver,Normal,Crossing from driver's nearside,Changing lane to the right,Slight Injury
9,17:20:00,Friday,18-30,Male,Junior high school,Employee,1-2yr,Automobile,Owner,2-5yrs,No defect,Residential areas,Undivided Two way,Tangent road with flat terrain,Y Shape,Asphalt roads,Dry,Daylight,Normal,Collision with roadside-parked vehicles,2,1,U-Turn,Passenger,Male,18-30,3,Driver,Normal,Not a Pedestrian,Moving Backward,Serious Injury


##3. Dealing with High Cardinality and Data Types

First I convert `Time` column to datetime and only extract the hours.

In [14]:
# Convert the Time column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Extract the hour from the datetime and assign it back to the Time column
df['Time'] = df['Time'].dt.hour

Removing duplicate categories and grouping categories

In [5]:
#Education Level
df['Educational_level'] = df['Educational_level'].replace('unknown', 'Unknown')

#Owner_of_vehicle
df['Owner_of_vehicle'] = df['Owner_of_vehicle'].replace('Other', 'unknown')

#Road_surface_conditions
df['Road_surface_conditions'] = df['Road_surface_conditions'].replace('Flood over 3cm. deep', 'Flood')

#Road_surface_conditions
df['Driving_experience'] = df['Driving_experience'].replace('nan', 'unknown')

In [6]:

# Replace 'Public (> 45 seats)', 'Public (12 seats)', and 'Public (13?45 seats)' with 'Public'
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Public (> 45 seats)', 'Public (12 seats)', 'Public (13?45 seats)'], 'Public')

# Replace 'Lorry (11?40Q)', 'Long lorry', and 'Lorry (41?100Q)' with 'Lorry'
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Lorry (11?40Q)', 'Long lorry', 'Lorry (41?100Q)'], 'Lorry')

# Replace 'unknown' with 'Unknown'
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace('unknown', 'Unknown')

# Combine 'Taxi', 'Stationwagen', and 'Automobile' into 'Automobile'
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Taxi', 'Stationwagen', 'Automobile'], 'Automobile')

# Combine 'Ridden horse', 'Bajaj', 'Special vehicle', and 'Turbo' into 'Other'
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace(['Ridden horse', 'Bajaj', 'Special vehicle', 'Turbo'], 'Other')


In [7]:
# Cause_of_accident
cause_mapping = {
    'Moving Backward': 'Reckless driving',
    'Overtaking': 'Reckless driving',
    'Changing lane to the left': 'Reckless driving',
    'Changing lane to the right': 'Reckless driving',
    'Overloading': 'Vehicle condition',
    'No priority to vehicle': 'Traffic rules violation',
    'No priority to pedestrian': 'Traffic rules violation',
    'No distancing': 'Traffic rules violation',
    'Getting off the vehicle improperly': 'Improper driving',
    'Improper parking': 'Improper driving',
    'Overspeed': 'Reckless driving',
    'Driving carelessly': 'Reckless driving',
    'Driving at high speed': 'Reckless driving',
    'Driving to the left': 'Reckless driving',
    'Unknown': 'Other',
    'Overturning': 'Vehicle condition',
    'Turnover': 'Vehicle condition',
    'Driving under the influence of drugs': 'Driving under influence',
    'Drunk driving': 'Driving under influence'
}

df['Cause_of_accident'] = df['Cause_of_accident'].map(cause_mapping)

In [8]:
# Pedestrian_movement
Pedestrian_movement_mapping = {
    'Not a Pedestrian': 'Other',
    "Crossing from driver's nearside": 'Crossing',
    'Crossing from nearside - masked by parked or statioNot a Pedestrianry vehicle': 'Crossing',
    'Unknown or other': 'Other',
    'Crossing from offside - masked by  parked or statioNot a Pedestrianry vehicle': 'Crossing',
    'In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing)': 'Other',
    'Walking along in carriageway, back to traffic': 'Other',
    'Walking along in carriageway, facing traffic': 'Other',
    'In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing) - masked by parked or statioNot a Pedestrianry vehicle': 'Other'
}

df['Pedestrian_movement_category'] = df['Pedestrian_movement'].map(Pedestrian_movement_mapping)


In [9]:
# Type_of_collision
type_collision_mapping = {'Collision with roadside-parked vehicles': 'Collision with roadside',
                          'Collision with roadside objects': 'Collision with roadside',
                          'Collision with pedestrians': 'Pedestrian collision',
                          'With Train': 'Train collision',
                          'unknown': 'Unknown'}

# Applying the mapping to the feature column
df['Type_of_collision'] = df['Type_of_collision'].map(type_collision_mapping)


In [10]:
# Road_allignment
road_allignment_mapping = {'Tangent road with flat terrain': 'Flat terrain tangent road',
                           'Tangent road with mild grade and flat terrain': 'Flat terrain tangent road',
                           'Tangent road with rolling terrain': 'Flat terrain tangent road',
                           'Tangent road with mountainous terrain and': 'Mountainous terrain tangent road',
                           'Steep grade downward with mountainous terrain': 'Mountainous terrain tangent road',
                           'unknown': 'Unknown'}

# Applying the mapping to the feature column
df['Road_allignment'] = df['Road_allignment'].map(road_allignment_mapping)


In [11]:
# Lanes_or_Medians
lanes_mapping = {'Double carriageway (median)': 'Divided highway',
                 'Two-way (divided with solid lines road marking)': 'Divided highway',
                 'Two-way (divided with broken lines road marking)': 'Divided highway',
                 'Undivided Two way': 'Undivided road',
                 'One way': 'Undivided road',
                 'other': 'Other',
                 'unknown': 'Unknown'}

# Applying the mapping to the feature column
df['Lanes_or_Medians'] = df['Lanes_or_Medians'].map(lanes_mapping)


Removing unrelated columns

In [ ]:
df = df.drop(['Fitness_of_casuality', 'Work_of_casuality', 'Area_accident_occured'], axis=1)

In [16]:
df.head(2)

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Pedestrian_movement,Cause_of_accident,Accident_severity,Pedestrian_movement_category
0,17,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,No defect,NaN,Flat terrain tangent road,No junction,Asphalt roads,Dry,Daylight,Normal,Collision with roadside,2,2,Going straight,na,na,na,na,Not a Pedestrian,Reckless driving,Slight Injury,Other
1,17,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public,Owner,5-10yrs,No defect,Undivided road,Flat terrain tangent road,No junction,Asphalt roads,Dry,Daylight,Normal,NaN,2,2,Going straight,na,na,na,na,Not a Pedestrian,Reckless driving,Slight Injury,Other


In [21]:
df['Educational_level'].fillna('unknown', inplace=True)
df['Vehicle_driver_relation'].fillna('unknown', inplace=True)
df['Driving_experience'].fillna('unknown', inplace=True)
df['Type_of_vehicle'].fillna('unknown', inplace=True)
df['Cause_of_accident'].fillna('unknown', inplace=True)
df['Owner_of_vehicle'].fillna('unknown', inplace=True)
df['Service_year_of_vehicle'].fillna('unknown', inplace=True)
df['Defect_of_vehicle'].fillna('unknown', inplace=True)
df['Lanes_or_Medians'].fillna('unknown', inplace=True)
df['Road_allignment'].fillna('unknown', inplace=True)
df['Types_of_Junction'].fillna('unknown', inplace=True)
df['Road_surface_type'].fillna('unknown', inplace=True)
df['Type_of_collision'].fillna('unknown', inplace=True)
df['Vehicle_movement'].fillna('unknown', inplace=True)

In [22]:
inference = df.sample(3)

In [23]:
inference

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,Lanes_or_Medians,Road_allignment,Types_of_Junction,Road_surface_type,Road_surface_conditions,Light_conditions,Weather_conditions,Type_of_collision,Number_of_vehicles_involved,Number_of_casualties,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Pedestrian_movement,Cause_of_accident,Accident_severity,Pedestrian_movement_category
9155,2,Sunday,18-30,Male,Junior high school,Employee,2-5yr,Other,Governmental,5-10yrs,No defect,Undivided road,Flat terrain tangent road,Y Shape,Asphalt roads,Dry,Darkness - lights lit,Normal,unknown,1,2,Moving Backward,na,na,na,na,Not a Pedestrian,Traffic rules violation,Slight Injury,Other
1656,19,Saturday,Unknown,Male,Junior high school,Employee,2-5yr,Lorry,Owner,5-10yrs,unknown,Undivided road,Flat terrain tangent road,Y Shape,Asphalt roads,Dry,Darkness - lights lit,Normal,unknown,2,1,Moving Backward,Passenger,Female,Under 18,3,Not a Pedestrian,Reckless driving,Slight Injury,Other
11121,18,Tuesday,18-30,Unknown,Elementary school,Employee,2-5yr,Automobile,unknown,unknown,unknown,Divided highway,Flat terrain tangent road,unknown,Asphalt roads,Dry,Daylight,Normal,unknown,1,1,Going straight,Pedestrian,Male,31-50,3,Unknown or other,Reckless driving,Slight Injury,Other


##4. Importing Models

In [46]:
# Load the saved model using joblib
loaded_model3 = joblib.load('/content/decision_tree_best.pkl')
loaded_model2 = joblib.load('/content/standard_scaler (2).pkl')
loaded_model1 = joblib.load('/content/encoder (1).pkl')

##5. Feature Engineering and Predicting

Encoding and Scaling

In [82]:
object_cols = inference[['Day_of_week', 'Age_band_of_driver', 'Sex_of_driver', 'Educational_level', 'Vehicle_driver_relation', 'Driving_experience', 'Type_of_vehicle', 'Owner_of_vehicle', 'Service_year_of_vehicle', 'Defect_of_vehicle', 'Lanes_or_Medians', 'Road_allignment', 'Types_of_Junction', 'Road_surface_type', 'Road_surface_conditions', 'Light_conditions', 'Weather_conditions', 'Type_of_collision', 'Vehicle_movement', 'Casualty_class', 'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity', 'Pedestrian_movement', 'Cause_of_accident', 'Pedestrian_movement_category']]

In [83]:
# Extract the columns to be scaled
cols_to_scale = inference[['Time','Number_of_casualties', 'Number_of_vehicles_involved']]

In [84]:

# Create a boolean array to use as a condition for indexing
cond = df.columns.isin(object_cols)

# Encode the categorical columns of the DataFrame
encoded_cols = loaded_model1.transform(df.loc[:, cond])



In [86]:
# Scale the numerical columns of the DataFrame
scaled_cols = loaded_model2.transform(cols_to_scale)

In [ ]:
# Concatenate the encoded and scaled columns
combined_cols = np.concatenate((encoded_cols, scaled_cols), axis=1)

Predicting

In [ ]:
# Apply the prediction model to the new DataFrame
predictions1 = loaded_model3.predict(combined_cols)


# Add the predictions as a new column to the DataFrame
inference['target_prediction1'] = predictions1

In [90]:
inference['target_prediction1']

9155     0
1656     0
11121    0
Name: target_prediction1, dtype: int64